In [1]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib as plt


In [2]:
base_dir = "../../raw_data/"


## Trying the Poteaux csv files

In [9]:
def import_csv(tableau, files):
    dfs = []

    for i in files:
        csv_path = os.path.join(base_dir, f"maquette{i}", f"{tableau}{i}.csv")
        if os.path.exists(csv_path):
            # Cherche la ligne d'en-tête réelle
            with open(csv_path, encoding="utf-8") as f:
                for idx, line in enumerate(f):
                    if line.startswith("Id;"):
                        header_row = idx
                        break
            try:
                df = pd.read_csv(csv_path, sep=';', decimal=",", header=header_row)
                dfs.append(df)
                print(f"Chargé : {csv_path} ({df.shape[0]} lignes, {df.shape[1]} colonnes)")
                #print(df.head())
            except Exception as e:
                print(f"Erreur de parsing : {csv_path} -> {e}")
        else:
            print(f"Fichier non trouvé : {csv_path}")

    if dfs:
        dfs_concat = pd.concat(dfs, ignore_index=True)
        print(f"Total concaténé : {dfs_concat.shape[0]} lignes, {dfs_concat.shape[1]} colonnes")
    else:
        dfs_concat = pd.DataFrame()
        print("Aucun fichier murs.csv trouvé.")

    return dfs_concat


In [8]:
files = [1,2,3,4,5,6,11,12,13,14,15,16,17,18,19,20]
dfs_concat = import_csv('murs', files)

Chargé : ../../raw_data/maquette1/murs1.csv (1589 lignes, 146 colonnes)
Chargé : ../../raw_data/maquette2/murs2.csv (215 lignes, 149 colonnes)
Chargé : ../../raw_data/maquette3/murs3.csv (203 lignes, 143 colonnes)
Chargé : ../../raw_data/maquette4/murs4.csv (312 lignes, 96 colonnes)
Chargé : ../../raw_data/maquette5/murs5.csv (345 lignes, 94 colonnes)
Chargé : ../../raw_data/maquette6/murs6.csv (203 lignes, 91 colonnes)
Chargé : ../../raw_data/maquette11/murs11.csv (121 lignes, 115 colonnes)
Chargé : ../../raw_data/maquette12/murs12.csv (121 lignes, 118 colonnes)
Chargé : ../../raw_data/maquette13/murs13.csv (797 lignes, 104 colonnes)
Chargé : ../../raw_data/maquette14/murs14.csv (3518 lignes, 107 colonnes)
Chargé : ../../raw_data/maquette15/murs15.csv (1228 lignes, 113 colonnes)
Chargé : ../../raw_data/maquette16/murs16.csv (1088 lignes, 87 colonnes)
Chargé : ../../raw_data/maquette17/murs17.csv (84 lignes, 86 colonnes)
Chargé : ../../raw_data/maquette18/murs18.csv (1743 lignes, 91 co

### FEATURES SELECTION:

In [19]:
all_features = dfs_concat.columns.to_list()

In [27]:
percent_missing = dfs_concat.isnull().sum() * 100 / len(dfs_concat)

missing_value_df = pd.DataFrame({'column_name': dfs_concat.columns,
                                 'percent_missing': percent_missing})

In [32]:
feature_list = missing_value_df.sort_values('percent_missing',
                                            ascending=False)['column_name'].to_list()


In [86]:
selected_features_list = [
    "011EC_Lot",
    "012EC_Ouvrage",
    "013EC_Localisation",
    "014EC_Mode Constructif",
    "Epaisseur",
    "Sols en intersection",
    "Sols coupés (u)",
    "Sols coupants (u)",
    "Sol au-dessus",
    "Sol en-dessous",
    "Fenêtres",
    "Portes",
    "Ouvertures",
    #"Murs imbriqués",
    "Mur multicouche",
    "Profil modifié",
    #"Extension inférieure",
    #"Extension supérieure",
    "Partie inférieure attachée",
    "Partie supérieure attachée",
    "Décalage supérieur",
    "Décalage inférieur",
    "Matériau structurel",
    "Famille et type"
]

targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

df_clean = dfs_concat[selected_features_list].dropna(axis=0, how='any')
df_clean.shape

(13508, 21)

In [87]:
#ignoring '011EC_Lot' for now / all GO
X = df_clean.drop(columns = ['012EC_Ouvrage','013EC_Localisation','014EC_Mode Constructif','011EC_Lot'])
y = df_clean[['012EC_Ouvrage','013EC_Localisation','014EC_Mode Constructif']]
y.shape, X.shape

((13508, 3), (13508, 17))

In [88]:
X.head(5)

,Epaisseur,Sols en intersection,Sols coupés (u),Sols coupants (u),Sol au-dessus,Sol en-dessous,Fenêtres,Portes,Ouvertures,Mur multicouche,Profil modifié,Partie inférieure attachée,Partie supérieure attachée,Décalage supérieur,Décalage inférieur,Matériau structurel,Famille et type
0,0.2,0,0,3,True,True,0,0,0,False,True,0,0,0.0,0.0,ECSA - Béton Voiles,Mur de base: Voile BA 20
1,0.2,0,0,1,False,True,0,0,0,False,True,0,0,0.0,0.0,ECSA - Béton Voiles,Mur de base: Voile BA 20
2,0.2,0,0,3,True,True,0,1,0,False,True,0,0,0.0,0.0,ECSA - Béton Voiles,Mur de base: Voile BA 20
3,0.2,0,0,3,True,True,0,1,0,False,True,0,0,0.0,0.0,ECSA - Béton Voiles,Mur de base: Voile BA 20
4,0.2,0,0,3,False,True,0,0,0,False,True,0,0,0.0,0.0,ECSA - Béton Voiles,Mur de base: Voile BA 20


In [89]:
y.sample(4)

,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif
5705,MUR,INTERIEUR,BANCHE
287,MUR,EXTERIEUR,POUTRE VOILE
9957,MUR,JD,PREMUR
6392,MUR,INTERIEUR,PREMUR


In [90]:
for feat in X.columns:
    print(f'the uniques for {feat}: {len(X[feat].unique())}')


the uniques for Epaisseur: 29
the uniques for Sols en intersection: 7
the uniques for Sols coupés (u): 2
the uniques for Sols coupants (u): 11
the uniques for Sol au-dessus: 2
the uniques for Sol en-dessous: 2
the uniques for Fenêtres: 13
the uniques for Portes: 11
the uniques for Ouvertures: 3
the uniques for Mur multicouche: 2
the uniques for Profil modifié: 2
the uniques for Partie inférieure attachée: 2
the uniques for Partie supérieure attachée: 2
the uniques for Décalage supérieur: 532
the uniques for Décalage inférieur: 343
the uniques for Matériau structurel: 25
the uniques for Famille et type: 143


In [94]:
for feat in X.columns:
    print(f'the uniques for {feat}: {len(X[feat].unique())} || They are: \n {X[feat].unique()}')


the uniques for Epaisseur: 29 || They are: 
 [0.2  0.3  0.16 0.18 0.25 0.15 0.4  0.35 0.47 0.67 0.17 1.01 0.61 1.28
 0.88 1.51 0.21 0.33 0.1  0.06 0.01 0.22 0.12 0.53 0.34 0.28 0.24 0.26
 0.5 ]
the uniques for Sols en intersection: 7 || They are: 
 [0 1 2 3 4 9 8]
the uniques for Sols coupés (u): 2 || They are: 
 [0 1]
the uniques for Sols coupants (u): 11 || They are: 
 [ 3  1  2  4  5  6  7  0  8 10  9]
the uniques for Sol au-dessus: 2 || They are: 
 [1 0]
the uniques for Sol en-dessous: 2 || They are: 
 [1 0]
the uniques for Fenêtres: 13 || They are: 
 [ 0  1 15 13  2  3  4  8  5 18  6 10  7]
the uniques for Portes: 11 || They are: 
 [ 0  1  2  9  8  3  4  7  6  5 24]
the uniques for Ouvertures: 3 || They are: 
 [0 1 2]
the uniques for Mur multicouche: 2 || They are: 
 [0 1]
the uniques for Profil modifié: 2 || They are: 
 [1 0]
the uniques for Partie inférieure attachée: 2 || They are: 
 [0 1]
the uniques for Partie supérieure attachée: 2 || They are: 
 [0 1]
the uniques for Décala

In [95]:
#Boolean to 0/1 for
bool_feats = ['Sol au-dessus', 'Sol en-dessous','Mur multicouche','Profil modifié']
for feat in bool_feats:
    X[feat] = X[feat].apply(lambda x: 1 if x == True else 0)

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features)
    ]
)

# Pipeline complet avec MultiOutputClassifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MultiOutputClassifier(RandomForestClassifier(n_estimators=5000, random_state=42)))
])


In [55]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(X[['Matériau structurel']])
ohe.categories_

[array(['.EC.R Béton TOUS chantier', '<Par catégorie>', 'BRIQUES',
        'BRIQUES RAL 1013', 'Blocs béton manufacturés', 'BÉTON', 'C25/30',
        'C25/30 - béton standard', 'C25/30 - peaux extérieures prémurs',
        'C25/30 - éléments préfa', 'EC - Béton', 'ECSA - Béton Voiles',
        'EIF_STR - BETON', 'GOE-Béton Coulé sur Place',
        'GOE-Béton Coulé sur Place-VNP',
        'GOE-Béton Coulé sur place-1ere PHASE', 'GOE-Parpaings Creux',
        'GOE-Parpaings Pleins', 'Maçonnerie',
        'Maçonnerie - Parpaing creux', 'Maçonnerie - Parpaing plein',
        'Maçonnerie - Voile BA', 'TPFI_BA', 'TPFI_BOIS', 'Verre'],
       dtype=object)]

In [ ]:
labeler2 = {'POTEAU RECTANGULAIRE':0,
           'POTEAU PARTICULIER':1,
           'POTEAU CIRCULAIRE':2,
           'MUR':3,
}
y['ouvrage'] = y['012EC_Ouvrage'].apply(lambda x: labeler2[x])

/tmp/ipykernel_16994/1705354856.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['ouvrage'] = y['012EC_Ouvrage'].apply(lambda x: labeler2[x])


In [147]:
feats = ['Matériau structurel','Famille et type']
for feat in feats:
    print(f'Transforming {feat}: {len(X[feat].unique())} values')
    ohe = OneHotEncoder(sparse_output=False)
    ohe.fit(X[[feat]])
    X[ohe.get_feature_names_out()] = ohe.transform(X[[feat]])
    X.drop(columns=[feat], inplace=True)


Transforming Matériau structurel: 6 values
Transforming Famille et type: 30 values


In [148]:
X.shape

(519, 40)

In [149]:
y['012EC_Ouvrage'].unique()

array(['POTEAU RECTANGULAIRE', 'POTEAU PARTICULIER', 'POTEAU CIRCULAIRE',
       'MUR'], dtype=object)

In [151]:
y['013EC_Localisation'].unique()

array(['INTERIEUR', 'EXTERIEUR', 'COURANT', 'BASSINS PERIPHERIES'],
      dtype=object)

In [152]:
labeler2 = {'INTERIEUR':0,
           'EXTERIEUR':1,
           'COURANT':2,
           'BASSINS PERIPHERIES':3,
}
y['local'] = y['013EC_Localisation'].apply(lambda x: labeler2[x])

/tmp/ipykernel_16994/2827905674.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['local'] = y['013EC_Localisation'].apply(lambda x: labeler2[x])


In [153]:
y['014EC_Mode Constructif'].unique()

array(['BANCHE', 'PREFA CHANTIER', 'PREFA USINE', 'CARTON'], dtype=object)

In [154]:
y['014EC_Mode Constructif'].unique()
labeler1 = {'BANCHE':0,
           'PREFA CHANTIER':1,
           'PREFA USINE':2,
           'CARTON':3,
       }
y['const'] = y['014EC_Mode Constructif'].apply(lambda x: labeler1[x])

/tmp/ipykernel_16994/1203657966.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['const'] = y['014EC_Mode Constructif'].apply(lambda x: labeler1[x])


In [139]:
y.sample(1)

,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,local,Ouvrage,construc
515,POTEAU RECTANGULAIRE,INTERIEUR,CARTON,0,0,3


In [164]:
from sklearn.dummy import DummyClassifier


dummy_clf_local = DummyClassifier(strategy="stratified")
dummy_clf_local.fit(X, y.local)


DummyClassifier(strategy='stratified')

In [165]:
set(dummy_clf_local.predict(X))


{0, 1, 2, 3}

In [ ]:
dummy_clf_local.score(X, y.local)

0.49710982658959535

In [191]:
dummy_clf_construc = DummyClassifier(strategy="stratified")
dummy_clf_construc.fit(X, y.const)

DummyClassifier(strategy='stratified')

In [192]:
set(dummy_clf_construc.predict(X))

{0, 1, 2, 3}

In [193]:
dummy_clf_construc.score(X, y.const)

0.35645472061657035

In [180]:
dummy_clf_ouvrage = DummyClassifier(strategy="stratified")
dummy_clf_ouvrage.fit(X, y.ouvrage)

DummyClassifier(strategy='stratified')

In [187]:
y_pred = dummy_clf_ouvrage.predict(X)

In [189]:
dummy_clf_ouvrage.score(y_pred, y.const)

0.35260115606936415